In [1]:
import requests
import json, re, os, sys

In [2]:
BASE_SISTR_URL = 'https://lfz.corefacility.ca/sistr-wtf/api/'

### Try to access user info with invalid creds

SISTR should return a 403 and some JSON data saying 'Unauthorized access'.

In [6]:
username = 'peter'
pw = '1'

In [7]:
r = requests.get('{}user/{}'.format(BASE_SISTR_URL, username), auth=(username, pw))

In [10]:
type(r)

requests.models.Response

In [8]:
r.status_code

403

In [9]:
r.json()

{u'error': u'Unauthorized access'}

## Create a new anon user by POSTing to /api/user/sistr

SISTR will automatically create a random username and return some basic user info and HTTP status code 201.

In [11]:
r = requests.post('{}user/sistr'.format(BASE_SISTR_URL))

In [12]:
r.status_code

201

In [13]:
user_info = r.json()

In [14]:
user_info

{u'last_seen': u'2016-04-01T19:03:33.825076',
 u'name': u'vNLjV66yMa60H2R1hJFp',
 u'role': u'temporary'}

In [15]:
username = user_info['name']

In [16]:
username

u'vNLjV66yMa60H2R1hJFp'

## Prep a Salmonella genome FASTA for upload

In [34]:
genome_fasta_path = !ls /home/peter/Downloads/14106.fasta

In [35]:
genome_fasta_path = genome_fasta_path[0]

In [36]:
genome_fasta_path

'/home/peter/Downloads/14106.fasta'

In [37]:
genome_name = os.path.basename(genome_fasta_path).replace('.fasta', '')

In [38]:
genome_name

'14106'

## Python requests needs a dict of POST params to binary read file handles

SISTR expects the FASTA file to be attached to `fasta` POST key param.

In [39]:
files = {'fasta': open(genome_fasta_path, 'rb')}

In [40]:
rpost = requests.post('{}user/{}/genome/{}'.format(BASE_SISTR_URL, username, genome_name), files=files)

In [41]:
genome_upload_resp = rpost.json()

POST response returns the SISTR genome name, task id to check for analysis status and the username

In [42]:
genome_upload_resp

{u'genome': u'14106_2',
 u'task': u'b528a667-56e4-44dd-a715-323d9da09405',
 u'user': u'vNLjV66yMa60H2R1hJFp'}

In [186]:
r_task = requests.get(BASE_SISTR_URL + 'task', params={'task_id': genome_upload_resp['task']})

In [187]:
r_task.json()

[{u'info': True,
  u'result': True,
  u'status': u'SUCCESS',
  u'task_id': u'b528a667-56e4-44dd-a715-323d9da09405'}]

In [182]:
r_results = requests.get('{}user/{}/genome/{}/serovar_prediction'.format(BASE_SISTR_URL, username, genome_upload_resp['genome']))

In [183]:
r_results.json()

{u'H1_prediction': u'i',
 u'H2_prediction': u'1,5',
 u'MLST_ST_in_silico': u'617',
 u'MLST_serovar_count_predictions': u'Cotham (2/2)',
 u'MLST_serovar_prediction': u'Cotham',
 u'Serogroup_prediction': u'M',
 u'cgMLST_cluster_level': u'0.65',
 u'cgMLST_cluster_number': 29,
 u'cgMLST_serovar_count_predictions': u'Poona (19/19)',
 u'cgMLST_serovar_prediction': u'Poona',
 u'curated_serovar': None,
 u'curation_info': None,
 u'genome': u'14106_2',
 u'reported_serovar': None,
 u'serovar_antigen_prediction': u'Cotham',
 u'serovar_prediction': u'Cotham',
 u'user_uploader': u'vNLjV66yMa60H2R1hJFp'}

In [188]:
r_mist_results = requests.get('{}user/{}/genome/{}/mist_results'.format(BASE_SISTR_URL, username, genome_upload_resp['genome']))

In [189]:
r_mist_results.json()

{u'genome': u'14106_2',
 u'marker_results': {u'MLST': {u'AROC': {u'has_potential_homopolymer_errors': False,
    u'is_contig_truncated': False,
    u'is_missing': False,
    u'result': u'13',
    u'time_added': u'Sat, 02 Apr 2016 06:40:28 -0000'},
   u'DNAN': {u'has_potential_homopolymer_errors': False,
    u'is_contig_truncated': False,
    u'is_missing': False,
    u'result': u'181',
    u'time_added': u'Sat, 02 Apr 2016 06:40:28 -0000'},
   u'HEMD': {u'has_potential_homopolymer_errors': False,
    u'is_contig_truncated': False,
    u'is_missing': False,
    u'result': u'49',
    u'time_added': u'Sat, 02 Apr 2016 06:40:28 -0000'},
   u'HISD': {u'has_potential_homopolymer_errors': False,
    u'is_contig_truncated': False,
    u'is_missing': False,
    u'result': u'13',
    u'time_added': u'Sat, 02 Apr 2016 06:40:28 -0000'},
   u'PURE': {u'has_potential_homopolymer_errors': False,
    u'is_contig_truncated': False,
    u'is_missing': False,
    u'result': u'183',
    u'time_added': u'S